# Channel Pruning on CIFAR-10

## 1. Phương pháp

### Phương pháp 1:L1-norm / L2-norm Filter Pruning

**Ý tưởng:**
Trong CNN, mỗi filter (kernel) sinh ra một feature map (output channel).
Độ quan trọng của filter có thể được ước lượng thông qua magnitude (độ lớn) của các trọng số trong filter.

Nếu norm nhỏ → filter ít đóng góp → có thể loại bỏ.

Nếu norm lớn → filter quan trọng → giữ lại.
**Đo độ quan trọng:**
***Với filter $F \in \mathbb{R}^{C \times k \times k}$:***
***L1-norm***
$$
\|F\|_{1} = \sum_{c=1}^C \sum_{i=1}^k \sum_{j=1}^k \left| F_{c,i,j} \right|
$$
***L2-norm***
$$
\|F\|_{2} = \sqrt{ \sum_{c=1}^C \sum_{i=1}^k \sum_{j=1}^k \left( F_{c,i,j} \right)^2 }
$$

**Thuật toán pruning:**

Tính L1 hoặc L2 norm cho tất cả các filters trong layer.

Sắp xếp filters theo norm tăng dần.

Loại bỏ một tỉ lệ $p%$ filters có norm thấp nhất.

Fine-tune lại mô hình để khôi phục accuracy.


### Phương pháp 2:BatchNorm Scaling Factor Pruning (Network Slimming)

**Ý tưởng:**

Trong CNN, mỗi kênh (channel) thường đi kèm một lớp Batch Normalization (BN).
BN có tham số scale $\gamma$ điều chỉnh độ quan trọng của channel.

Nếu $\gamma$ nhỏ → channel đóng góp ít → có thể loại bỏ.

Nếu $\gamma$ lớn → channel quan trọng → giữ lại.
**Cơ chế chính:**
Thêm regularization L1 trực tiếp lên các tham số $\gamma$ trong quá trình huấn luyện:

$$
\mathcal{L} = \mathcal{L}_{\text{task}} + \lambda \sum |\gamma|
$$


Trong đó:

$\mathcal{L}_{\text{task}}$: loss gốc (ví dụ cross-entropy).

$\lambda$: hệ số điều chỉnh độ sparsity.

**Thuật toán pruning:**

Huấn luyện mô hình với regularization L1 lên $\gamma$.

Sau khi huấn luyện, nhiều $\gamma$ sẽ tiến gần về 0.

Chọn ngưỡng $\tau$, loại bỏ các channel có $|\gamma| < \tau$.

Fine-tune lại mô hình để phục hồi accuracy.

### Phương pháp 3:Activation-based Pruning

**Ý tưởng:**

Trong CNN, mức độ quan trọng của một filter có thể được đánh giá dựa trên activation (đầu ra feature map) của filter đó.

Nếu một filter thường xuyên tạo ra activation yếu hoặc gần như bằng 0 → nó ít đóng góp thông tin → có thể loại bỏ.

Nếu activation mạnh, đa dạng → filter quan trọng → giữ lại.

**Cách đo độ quan trọng:**

Với feature map $A \in \mathbb{R}^{H \times W}$ được sinh ra bởi filter $F$:

***Average Activation Magnitude***
$$
I(F) = \frac{1}{N \cdot H \cdot W} \sum_{n=1}^N \sum_{i=1}^H \sum_{j=1}^W |A_{n,i,j}|
$$

***Variance of Activation***
$$
I(F) = \mathrm{Var}(A)
$$


Trong đó $N$ là số lượng mẫu (mini-batch).

**Thuật toán pruning:**

Chạy dữ liệu qua mạng để thu thập activation cho từng filter.

Tính chỉ số quan trọng $I(F)$ theo trung bình hoặc phương sai activation.

Sắp xếp filters theo $I(F)$ tăng dần.

Loại bỏ một tỉ lệ $p%$ filters có activation yếu nhất.

Fine-tune mô hình để khôi phục accuracy.

### Phương pháp 4:Taylor Expansion / Gradient-based Pruning

**Ý tưởng:**
Mức độ quan trọng của một filter có thể được ước lượng bằng cách xem xét tác động của việc loại bỏ filter đó đến loss.
Sử dụng khai triển Taylor bậc 1, xấp xỉ được sự thay đổi loss khi một trọng số hoặc filter bị loại bỏ.

**Công thức chính:**
***Với một trọng số $w$, thay đổi loss khi áp dụng $\Delta w$ được xấp xỉ bởi:***
$$
\Delta \mathcal{L} \approx \frac{\partial \mathcal{L}}{\partial w} \cdot \Delta w
$$
***Khi prune bằng cách đặt $w = 0$ thì $\Delta w = -w$, do đó:***
$$
\Delta \mathcal{L} \approx - \frac{\partial \mathcal{L}}{\partial w} \cdot w
$$
***Độ quan trọng của một trọng số $w$:***
$$
I(w) = \left| \frac{\partial \mathcal{L}}{\partial w} \cdot w \right|
$$
***Với một filter $F \in \mathbb{R}^{C \times k \times k}$, độ quan trọng được tính bằng:***
$$
I(F) = \mathbb{E}_{(x,y)\sim \mathcal{D}} \left[ \left| \sum_{c=1}^C \sum_{i=1}^k \sum_{j=1}^k \frac{\partial \mathcal{L}}{\partial F_{c,i,j}} \cdot F_{c,i,j} \right| \right]
$$

**Thuật toán pruning:**

Chạy một số batch dữ liệu qua mạng và tính gradient $\frac{\partial \mathcal{L}}{\partial F}$ cho mỗi filter $F$.

Tính chỉ số quan trọng $I(F)$ cho từng filter.

Sắp xếp các filter theo $I(F)$ tăng dần.

Loại bỏ một tỉ lệ $p%$ filter có $I(F)$ nhỏ nhất.

Fine-tune mô hình đã prune để khôi phục accuracy.


## 2. Đánh giá